# Sprint 深層学習スクラッチ リカレントニューラルネットワーク

## 1.このSprintについて

### Sprintの目的
- スクラッチを通してリカレントニューラルネットワークの基礎を理解する

### どのように学ぶか
スクラッチでリカレントニューラルネットワークの実装を行います。

## 2.リカレントニューラルネットワークスクラッチ

**リカレントニューラルネットワーク（RNN）** のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。

クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

### 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

バッチサイズを**batch_size**、入力の特徴量数を**n_features**、RNNのノード数を**n_nodes**として表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\
h_t = tanh(a_t)
$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)

$x_t$ : 時刻tの入力 (batch_size, n_features)

$W_x$ : 入力に対する重み (n_features, n_nodes)

$h_t-1$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)

$W_h$ : 状態に対する重み。 (n_nodes, n_nodes)

$B$ : バイアス項 (n_nodes,)

初期状態 $h_0$ は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
$x$
 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [1]:
import numpy as np

In [2]:
class SimpleCell:
    """
    RNNそれぞれのセル
    Parameters
    ----------
    act_instance : 活性化関数のインスタンス
    """
    def __init__(self, act_instance):
        self.act_instance = act_instance
    def forward(self, X):
        self.X = X
        return X
    def backward(self, dh):
        return self.act_instance.backward(dh)

class SimpleRNN:
    """
    ノード数n_nodesのRNN層
    Parameters
    ----------
    n_sequences : int inputのデータ数
    n_features : int inputの特徴量数
    n_nodes : int RNNのノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    act_instance : 活性化関数のインスタンス（縦方向）
    cell_act_instance : cell内の活性化関数のインスタンス（横方向）
    """
    def __init__(self, n_sequences, n_features, n_nodes, initializer, optimizer, act_instance, cell_act_instance):
        self.n_sequences, self.n_features, self.n_nodes = n_sequences, n_features, n_nodes
        #　各セルのインスタンス
        self.cell_list = [SimpleCell(cell_act_instance) for _ in range(self.n_sequences)]
        # 最適化手法のインスタンス
        self.optimizer = optimizer
        # 活性化関数のインスタンス
        self.act_instance = act_instance
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        if initializer == None:
            # pass
            self.Wx = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
            self.Wh = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
            self.B = np.array([1, 1, 1, 1]) # (n_nodes,)
        else:
            self.Wx = initializer.W(n_features, n_nodes)
            self.Wh = initializer.W(n_nodes, n_nodes)
            self.B = initializer.B(n_nodes)
            # optimazer がadagradの際に使用する
            self.H_Wx = np.ones((n_features, n_nodes))
            self.H_Wx = np.ones((n_nodes, n_nodes))
            self.H_B = np.ones(n_nodes)

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_sequences, n_features)
        Returns
        ----------
        h_s : 次の形のndarray, shape (batch_size, n_sequences, n_nodes)
            A : 次の形のndarray, shape (batch_size, n_nodes)
                出力
        """
        self.X = X
        self.batch_size, _, _ = X.shape
        h_s = np.zeros((self.batch_size, self.n_sequences, self.n_nodes))
        
        Z = np.zeros((self.batch_size, self.n_nodes))
        for i in range(self.n_sequences):
            # print(self.X[:, i].shape, self.Wx.shape, np.dot(self.X[:, i], self.Wx).shape,"     ", self.Wh.shape, np.dot(self.cell_list[i].forward(Z), self.Wh).shape, "   ", self.B.shape)
            A = np.dot(self.X[:, i], self.Wx) + np.dot(self.cell_list[i].forward(Z), self.Wh) + self.B
            # print(A.shape)
            Z = self.cell_list[i].act_instance.forward(A)
            # print(Z.shape)
            h_s[:, i] = Z
        
        # 最終出力後、何らかの方法で誤差計算? 現状はゼロ
        self.dh = 0
        
        return h_s
    
    def backward(self, dh_s):
        """
        バックワード
        Parameters
        ----------
        dh_s : 次の形のndarray, shape (batch_size, n_sequences, n_nodes)
            後ろから流れてきた勾配
        Returns
        ----------
        dx : 次の形のndarray, shape (batch_size, n_sequences, n_features)
            前に流す勾配
        """
        # ベクトルの誤差を保管する
        dx = np.zeros((self.batch_size, self.n_sequences, self.n_features))
        # 各勾配を0で初期化
        self.dB, self.dWx, self.dWh = 0, 0, 0
        
        for i in range(self.n_sequences)[::-1]:
            dh_s[:, i] += self.dh
            dA = self.cell_list[i].backward(dh_s[:, i])
            #　前の時刻や層に流す誤差
            self.dh = np.dot(dA, self.Wh.T)
            dx[:, i] = np.dot(dA, self.Wx.T)
            
            # dB = dA 問題にシグマついていない？
            self.dB += dA.sum(axis=0)
            self.dWx += np.dot(self.X[:, i].T, dA)
            self.dWh += np.dot(self.cell_list[i].X.T, dA)
            
        # 勾配をもとに重みを更新
        self = self.optimizer.update_SimpleRNN(self, dA)
        
        return dx

##### class Optimizer

In [3]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr=0.001):
        self.lr = lr
    def update_SimpleRNN(self, layer, dA):
        """
        SimpleRNN層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        dA : 次の形のndarray, shape (batch_size, n_nodes)
            rnn層の最終出力からの勾配
        """
        # バイアス、重みの更新
        layer.Wx = layer.Wx - self.lr * layer.dWx
        layer.Wh = layer.Wh - self.lr * layer.dWh
        layer.B = layer.B - self.lr * layer.dB

##### class Initializer

In [4]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma=0.01):
        self.sigma = sigma
    def W(self, *parameters):
        """
        重みの初期化
        Parameters
        ----------
        Conv2d
            output_channels : int 出力チャネル数
            input_channels : int 入力チャネル数
            filter_sizes : tuple フィルター数
                filter_sizes_h, filter_sizes_w
        Conv1d
            output_channels : int 出力チャネル数
            input_channels : int 入力チャネル数
            filter_sizes : int フィルター数
        Affine
            n_nodes1 : int 前の層のノード数
            n_nodes2 : int 後の層のノード数
        Returns
        ----------
        W : shape(*parameters)
        """
        W = self.sigma * np.random.randn(*parameters)
        return W
    def B(self, output_channels):
        """
        バイアスの初期化
        Parameters
        ----------
        Conv2d
            output_channels : int 出力チャネル数
        Conv1d
            output_channels : int 出力チャネル数
        Affine
            n_node2 : int 後の層のノード数
        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(output_channels)
        return B

##### class Activation

In [5]:
class Sigmoid():
    def forward(self, A):
        self.A = A
        Z = 1.0 / (1.0 + np.exp(-self.A))
        return Z
    def backward(self, dZ):
        # dZとアダマール計算する値
        tmp =  (1.0 - self.forward(self.A)) * self.forward(self.A)
        dA = dZ * tmp
        return dA

class Tanh():
    def forward(self, A):
        self.A = A
        Z = np.tanh(self.A)
        return Z
    
    def backward(self, dZ):
        # dZとアダマール計算する値
        tmp = 1.0 - np.power(self.forward(self.A), 2)
        dA = dZ * tmp
        return dA

class ReLU():
    def forward(self, A):
        self.A = A
        Z = np.maximum(self.A, 0)
        return Z
    def backward(self, dZ):
        # dZとアダマール計算する値
        tmp = np.where(self.A > 0, 1, 0)
        dA = dZ * tmp
        return dA
    
class Softmax():
    def forward(self, A):
        self.A = A
        # オーバーフローを防ぐ
        self.A -= np.max(self.A)
        Z = np.exp(self.A) / np.exp(self.A).sum(axis=1)[:, np.newaxis]
        return Z

    def _cross_entropy_error(self, Z, y_label_one_hot):
        # error = -(np.log(Z) * y_label_one_hot).sum(axis=1).mean()
        error = -(np.log(Z + 10e-10) * y_label_one_hot).sum(axis=1).mean()
        return error

    def backward(self, Z, y_label_one_hot):
        # 交差エントロピー誤差の計算
        loss = self._cross_entropy_error(Z, y_label_one_hot)
        # dA　の計算
        dA = Z - y_label_one_hot
        # dA, loss
        return dA, loss

### 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。

入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。

ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [6]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。
```
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```

In [7]:
rnn = SimpleRNN(n_sequences, n_features, n_nodes, 
                initializer=None, optimizer=SGD(), act_instance=ReLU(), cell_act_instance=Tanh())
rnn.forward(x)

array([[[0.76188798, 0.76213958, 0.76239095, 0.76255841],
        [0.792209  , 0.8141834 , 0.83404912, 0.84977719],
        [0.79494228, 0.81839002, 0.83939649, 0.85584174]]])

### 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。

RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$
W_x^{\prime} = W_x - \alpha \frac{\partial L}{\partial W_x} \\
W_h^{\prime} = W_h - \alpha \frac{\partial L}{\partial W_h} \\
B^{\prime} = B - \alpha \frac{\partial L}{\partial B}
$$

$\alpha$ : 学習率

$\frac{\partial L}{\partial W_x}$
 : 
$W_x$ に関する損失 $L$ の勾配

$\frac{\partial L}{\partial W_h}$
: 
$W_h$ に関する損失 $L$ の勾配

$\frac{\partial L}{\partial B}$
: 
$B$ に関する損失 $L$ の勾配

勾配を求めるためのバックプロパゲーションの数式が以下です。

$\frac{\partial h_t}{\partial a_t} = \frac{\partial L}{\partial h_t} ×(1-tanh^2(a_t))$

$\frac{\partial L}{\partial B} = \frac{\partial h_t}{\partial a_t}$

$\frac{\partial L}{\partial W_x} = x_{t}^{T}\cdot \frac{\partial h_t}{\partial a_t}$

$\frac{\partial L}{\partial W_h} = h_{t-1}^{T}\cdot \frac{\partial h_t}{\partial a_t}$

＊
$\frac{\partial L}{\partial h_t}$
 は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。
 
 前の時刻や層に流す誤差の数式は以下です。
 
 $
 \frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_t}{\partial a_t}\cdot W_{h}^{T}
 $
 
 $
 \frac{\partial L}{\partial x_{t}} = \frac{\partial h_t}{\partial a_t}\cdot W_{x}^{T}
 $

In [8]:
dX = rnn.backward(rnn.forward(x))
dX, dX.shape

(array([[[0.03889262, 0.05730395],
         [0.04203426, 0.06159598],
         [0.03946   , 0.05796616]]]),
 (1, 3, 2))